# **Importing Libraries**

In [2]:
import pickle
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, Input, concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data Preprocessing**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving IndianFoodDatasetCSV.csv to IndianFoodDatasetCSV (1).csv


In [ ]:
data = pd.read_csv("/content/IndianFoodDatasetCSV.csv")
data = data.drop(["Srno", "RecipeName", "Ingredients", "Instructions", "URL"], axis=1)
data

,TranslatedRecipeName,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions
0,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se..."
1,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,South Indian Recipes,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato..."
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi..."
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Andhra,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ..."
...,...,...,...,...,...,...,...,...,...,...
6866,Goan Mushroom Xacuti Recipe,"20 बटन मशरुम,2 प्याज - काट ले,1 टमाटर - बारीक ...",15,45,60,4,Goan Recipes,Lunch,Vegetarian,गोअन मशरुम जकुटी रेसिपी बनाने के लिए सबसे पहले...
6867,Sweet Potato & Methi Stuffed Paratha Recipe,"1 बड़ा चम्मच तेल,1 कप गेहूं का आटा,नमक - स्वाद ...",30,60,90,4,North Indian Recipes,North Indian Breakfast,Diabetic Friendly,शकरकंदी और मेथी का पराठा रेसिपी बनाने के लिए स...
6868,Ullikadala Pulusu Recipe | Spring Onion Curry,150 grams Spring Onion (Bulb & Greens) - chopp...,5,10,15,2,Andhra,Side Dish,Vegetarian,To begin making Ullikadala Pulusu Recipe | Spr...
6869,Kashmiri Style Kokur Yakhni Recipe-Chicken Coo...,"1 kg Chicken - medium pieces,1/2 cup Mustard o...",30,45,75,4,Kashmiri,Lunch,Non Vegeterian,To begin making the Kashmiri Kokur Yakhni reci...


In [ ]:
column_mapping = {'TranslatedRecipeName': 'Name', 'TranslatedIngredients': 'Ingredients', 'PrepTimeInMins': 'PrepTimeInMins', 'CookTimeInMins': 'CookTimeInMins', 'TotalTimeInMins': 'TotalTimeInMins', 'Servings': 'Servings', 'Cuisine': 'Cuisine', 'Course': 'Course', 'Diet': 'Diet', 'TranslatedInstructions': 'Instructions'}
data.rename(columns=column_mapping, inplace=True)
data

,Name,Ingredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,Instructions
0,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se..."
1,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,South Indian Recipes,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato..."
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi..."
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Andhra,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ..."
...,...,...,...,...,...,...,...,...,...,...
6866,Goan Mushroom Xacuti Recipe,"20 बटन मशरुम,2 प्याज - काट ले,1 टमाटर - बारीक ...",15,45,60,4,Goan Recipes,Lunch,Vegetarian,गोअन मशरुम जकुटी रेसिपी बनाने के लिए सबसे पहले...
6867,Sweet Potato & Methi Stuffed Paratha Recipe,"1 बड़ा चम्मच तेल,1 कप गेहूं का आटा,नमक - स्वाद ...",30,60,90,4,North Indian Recipes,North Indian Breakfast,Diabetic Friendly,शकरकंदी और मेथी का पराठा रेसिपी बनाने के लिए स...
6868,Ullikadala Pulusu Recipe | Spring Onion Curry,150 grams Spring Onion (Bulb & Greens) - chopp...,5,10,15,2,Andhra,Side Dish,Vegetarian,To begin making Ullikadala Pulusu Recipe | Spr...
6869,Kashmiri Style Kokur Yakhni Recipe-Chicken Coo...,"1 kg Chicken - medium pieces,1/2 cup Mustard o...",30,45,75,4,Kashmiri,Lunch,Non Vegeterian,To begin making the Kashmiri Kokur Yakhni reci...


In [ ]:
data.to_csv('IndianFoodDataset.csv', index=False)

# **Data Wrangling**

In [ ]:
value_counts_column1 = data['Diet'].value_counts()
value_counts_column2 = data['Cuisine'].value_counts()
value_counts_column3 = data['Course'].value_counts()

print("Value Counts for Diet:")
print("----------------------")
print(value_counts_column1)
print("---------------------------------------")
print("\nValue Counts for Cuisine:")
print("-------------------------")
print(value_counts_column2)
print("---------------------------------------")
print("\nValue Counts for Course:")
print("------------------------")
print(value_counts_column3)
print("---------------------------------------")

Value Counts for Diet:
----------------------
Vegetarian                      4712
High Protein Vegetarian          705
Non Vegeterian                   427
Eggetarian                       344
Diabetic Friendly                260
High Protein Non Vegetarian      225
No Onion No Garlic (Sattvic)      73
Vegan                             61
Gluten Free                       50
Sugar Free Diet                   14
Name: Diet, dtype: int64
---------------------------------------

Value Counts for Cuisine:
-------------------------
Indian                  1157
Continental             1021
North Indian Recipes     938
South Indian Recipes     682
Italian Recipes          236
                        ... 
Jewish                     1
Dessert                    1
Side Dish                  1
Shandong                   1
Lunch                      1
Name: Cuisine, Length: 82, dtype: int64
---------------------------------------

Value Counts for Course:
------------------------
Lunch           

In [ ]:
gujarati_rows = df[df['Cuisine'].str.startswith('Gujarati')]

# Remove special characters from the 'Cuisine' column in the filtered rows
gujarati_rows['Cuisine'] = gujarati_rows['Cuisine'].str.replace(r'[^a-zA-Z\s]', '')

# Update the original DataFrame with cleaned 'Cuisine' column
df.loc[gujarati_rows.index, 'Cuisine'] = gujarati_rows['Cuisine']

# Display the updated DataFrame
print("Updated DataFrame:")
print(df)

# Save the DataFrame to CSV
df.to_csv('/content/drive/MyDrive/IndianFoodDataset.csv', index=False)

<ipython-input-30-6d850369f61d>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  gujarati_rows['Cuisine'] = gujarati_rows['Cuisine'].str.replace(r'[^a-zA-Z\s]', '')
<ipython-input-30-6d850369f61d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gujarati_rows['Cuisine'] = gujarati_rows['Cuisine'].str.replace(r'[^a-zA-Z\s]', '')


Updated DataFrame:
                                                   Name  \
0                                  Masala Karela Recipe   
1                            Spicy Tomato Rice (Recipe)   
2     Ragi Semiya Upma Recipe - Ragi Millet Vermicel...   
3     Gongura Chicken Curry Recipe - Andhra Style Go...   
4     Andhra Style Alam Pachadi Recipe - Adrak Chutn...   
...                                                 ...   
6866                        Goan Mushroom Xacuti Recipe   
6867        Sweet Potato & Methi Stuffed Paratha Recipe   
6868      Ullikadala Pulusu Recipe | Spring Onion Curry   
6869  Kashmiri Style Kokur Yakhni Recipe-Chicken Coo...   
6870                                 Navrang Dal Recipe   

                                            Ingredients  PrepTimeInMins  \
0     6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...              15   
1     2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...               5   
2     1-1/2 cups Rice Vermicelli Noodles (Thin)

# **Model**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving IndianFoodDataset.csv to IndianFoodDataset.csv


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IndianFoodDataset.csv')

mlb = MultiLabelBinarizer()
categories = mlb.fit_transform(df['Diet'].apply(lambda x: x.split(', ')))
text_data_ingredients = df['Ingredients'].astype(str)
text_data_recipe_name = df['Name'].astype(str)

max_words = 1500
tokenizer_ingredients = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer_ingredients.fit_on_texts(text_data_ingredients)
sequences_ingredients = tokenizer_ingredients.texts_to_sequences(text_data_ingredients)
padded_sequences_ingredients = pad_sequences(sequences_ingredients, maxlen=150, padding='post', truncating='post')

tokenizer_recipe_name = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer_recipe_name.fit_on_texts(text_data_recipe_name)
sequences_recipe_name = tokenizer_recipe_name.texts_to_sequences(text_data_recipe_name)
padded_sequences_recipe_name = pad_sequences(sequences_recipe_name, maxlen=10, padding='post', truncating='post')

X_ingredients = padded_sequences_ingredients
X_recipe_name = padded_sequences_recipe_name
y = categories

X_train_ingredients, X_test_ingredients, X_train_recipe_name, X_test_recipe_name, y_train, y_test = train_test_split(
    X_ingredients, X_recipe_name, y, test_size=0.25, random_state=42
)

input_ingredients = Input(shape=(150,), name='Ingredients')
embedding_ingredients = Embedding(input_dim=max_words, output_dim=128, input_length=150)(input_ingredients)
lstm_ingredients = Bidirectional(LSTM(64, return_sequences=True))(embedding_ingredients)
lstm_ingredients = Bidirectional(LSTM(32))(lstm_ingredients)

input_recipe_name = Input(shape=(10,), name='recipe_name_input')
embedding_recipe_name = Embedding(input_dim=max_words, output_dim=128, input_length=10)(input_recipe_name)
lstm_recipe_name = Bidirectional(LSTM(32))(embedding_recipe_name)

merged = concatenate([lstm_ingredients, lstm_recipe_name])
output = Dense(categories.shape[1], activation='sigmoid')(merged)

model = Model(inputs=[input_ingredients, input_recipe_name], outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit([X_train_ingredients, X_train_recipe_name], y_train, epochs=10, validation_data=([X_test_ingredients, X_test_recipe_name], y_test))

loss, accuracy = model.evaluate([X_test_ingredients, X_test_recipe_name], y_test)
print(f'Test Accuracy: {accuracy}')

Epoch 1/10
162/162 [==============================] - 69s 357ms/step - loss: 0.2194 - accuracy: 0.6810 - val_loss: 0.1606 - val_accuracy: 0.6898
Epoch 2/10
162/162 [==============================] - 68s 421ms/step - loss: 0.1369 - accuracy: 0.7481 - val_loss: 0.1342 - val_accuracy: 0.7538
Epoch 3/10
162/162 [==============================] - 55s 341ms/step - loss: 0.1141 - accuracy: 0.8023 - val_loss: 0.1306 - val_accuracy: 0.7643
Epoch 4/10
162/162 [==============================] - 54s 335ms/step - loss: 0.0998 - accuracy: 0.8205 - val_loss: 0.1281 - val_accuracy: 0.7742
Epoch 5/10
162/162 [==============================] - 60s 371ms/step - loss: 0.0885 - accuracy: 0.8407 - val_loss: 0.1315 - val_accuracy: 0.7742
Epoch 6/10
162/162 [==============================] - 65s 400ms/step - loss: 0.0800 - accuracy: 0.8609 - val_loss: 0.1366 - val_accuracy: 0.7718
Epoch 7/10
162/162 [==============================] - 54s 331ms/step - loss: 0.0718 - accuracy: 0.8775 - val_loss: 0.1460 - val_ac

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IndianFoodDataset.csv')

mlb_cuisine = MultiLabelBinarizer()
categories_course = mlb_cuisine.fit_transform(df['Cuisine'].apply(lambda x: x.split(', ')))

max_words = 1500
tokenizer_ingredients = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer_ingredients.fit_on_texts(df['Ingredients'].astype(str))
sequences_ingredients = tokenizer_ingredients.texts_to_sequences(df['Ingredients'].astype(str))
padded_sequences_ingredients = pad_sequences(sequences_ingredients, maxlen=150, padding='post', truncating='post')

tokenizer_recipe_name = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer_recipe_name.fit_on_texts(df['Name'].astype(str))
sequences_recipe_name = tokenizer_recipe_name.texts_to_sequences(df['Name'].astype(str))
padded_sequences_recipe_name = pad_sequences(sequences_recipe_name, maxlen=10, padding='post', truncating='post')

X_ingredients = padded_sequences_ingredients
X_recipe_name = padded_sequences_recipe_name
y_course = categories_course

X_train_ingredients, X_test_ingredients, X_train_recipe_name, X_test_recipe_name, y_train_course, y_test_course = train_test_split(
    X_ingredients, X_recipe_name, y_course, test_size=0.25, random_state=42
)

input_ingredients = Input(shape=(150,), name='Ingredients')
embedding_ingredients = Embedding(input_dim=max_words, output_dim=128, input_length=150)(input_ingredients)
lstm_ingredients = Bidirectional(LSTM(64, return_sequences=True))(embedding_ingredients)
lstm_ingredients = Bidirectional(LSTM(32))(lstm_ingredients)

input_recipe_name = Input(shape=(10,), name='RecipeName')
embedding_recipe_name = Embedding(input_dim=max_words, output_dim=128, input_length=10)(input_recipe_name)
lstm_recipe_name = Bidirectional(LSTM(32))(embedding_recipe_name)

merged = concatenate([lstm_ingredients, lstm_recipe_name])
dropout = Dropout(0.2)(merged)
output_course = Dense(categories_course.shape[1], activation='sigmoid', name='output_course')(merged)

modelCu = Model(inputs=[input_ingredients, input_recipe_name], outputs=output_course)

modelCu.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
modelCu.fit([X_train_ingredients, X_train_recipe_name], y_train_course, epochs=20, validation_data=([X_test_ingredients, X_test_recipe_name], y_test_course))

loss, accuracy = modelCu.evaluate([X_test_ingredients, X_test_recipe_name], y_test_course)
print(f'Test Accuracy: {accuracy}')

Epoch 1/20
162/162 [==============================] - 67s 348ms/step - loss: 0.1268 - accuracy: 0.1230 - val_loss: 0.0488 - val_accuracy: 0.1834
Epoch 2/20
162/162 [==============================] - 55s 337ms/step - loss: 0.0499 - accuracy: 0.1587 - val_loss: 0.0488 - val_accuracy: 0.1356
Epoch 3/20
162/162 [==============================] - 54s 334ms/step - loss: 0.0496 - accuracy: 0.1644 - val_loss: 0.0483 - val_accuracy: 0.1834
Epoch 4/20
162/162 [==============================] - 61s 374ms/step - loss: 0.0487 - accuracy: 0.1840 - val_loss: 0.0469 - val_accuracy: 0.2078
Epoch 5/20
162/162 [==============================] - 55s 339ms/step - loss: 0.0465 - accuracy: 0.2408 - val_loss: 0.0446 - val_accuracy: 0.2986
Epoch 6/20
162/162 [==============================] - 63s 389ms/step - loss: 0.0427 - accuracy: 0.3272 - val_loss: 0.0410 - val_accuracy: 0.3702
Epoch 7/20
162/162 [==============================] - 62s 382ms/step - loss: 0.0384 - accuracy: 0.4347 - val_loss: 0.0378 - val_ac

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IndianFoodDataset.csv')

mlb_course = MultiLabelBinarizer()
categories_course = mlb_course.fit_transform(df['Course'].apply(lambda x: x.split(', ')))

max_words = 1500
tokenizer_ingredients = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer_ingredients.fit_on_texts(df['Ingredients'].astype(str))
sequences_ingredients = tokenizer_ingredients.texts_to_sequences(df['Ingredients'].astype(str))
padded_sequences_ingredients = pad_sequences(sequences_ingredients, maxlen=150, padding='post', truncating='post')

tokenizer_recipe_name = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer_recipe_name.fit_on_texts(df['Name'].astype(str))
sequences_recipe_name = tokenizer_recipe_name.texts_to_sequences(df['Name'].astype(str))
padded_sequences_recipe_name = pad_sequences(sequences_recipe_name, maxlen=10, padding='post', truncating='post')

tokenizer_instructions = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer_instructions.fit_on_texts(df['Instructions'].astype(str))
sequences_instructions = tokenizer_instructions.texts_to_sequences(df['Instructions'].astype(str))
padded_sequences_instructions = pad_sequences(sequences_instructions, maxlen=200, padding='post', truncating='post')

X_ingredients = padded_sequences_ingredients
X_recipe_name = padded_sequences_recipe_name
X_instructions = padded_sequences_instructions
y_course = categories_course

X_train_ingredients, X_test_ingredients, X_train_recipe_name, X_test_recipe_name, X_train_instructions, X_test_instructions, y_train_course, y_test_course = train_test_split(
    X_ingredients, X_recipe_name, X_instructions, y_course, test_size=0.25, random_state=42
)

input_ingredients = Input(shape=(150,), name='Ingredients')
embedding_ingredients = Embedding(input_dim=max_words, output_dim=128, input_length=150)(input_ingredients)
lstm_ingredients = Bidirectional(LSTM(64, return_sequences=True))(embedding_ingredients)
lstm_ingredients = Bidirectional(LSTM(32))(lstm_ingredients)

input_recipe_name = Input(shape=(10,), name='RecipeName')
embedding_recipe_name = Embedding(input_dim=max_words, output_dim=128, input_length=10)(input_recipe_name)
lstm_recipe_name = Bidirectional(LSTM(32))(embedding_recipe_name)

input_instructions = Input(shape=(200,), name='Instructions')
embedding_instructions = Embedding(input_dim=max_words, output_dim=128, input_length=200)(input_instructions)
lstm_instructions = Bidirectional(LSTM(64, return_sequences=True))(embedding_instructions)
lstm_instructions = Bidirectional(LSTM(32))(lstm_instructions)

merged = concatenate([lstm_ingredients, lstm_recipe_name, lstm_instructions])
dropout = Dropout(0.2)(merged)
output_course = Dense(categories_course.shape[1], activation='sigmoid', name='output_course')(dropout)

modelC = Model(inputs=[input_ingredients, input_recipe_name, input_instructions], outputs=output_course)

modelC.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
modelC.fit(
    [X_train_ingredients, X_train_recipe_name, X_train_instructions], y_train_course,
    epochs=15,
    validation_data=([X_test_ingredients, X_test_recipe_name, X_test_instructions], y_test_course),
    verbose=2
)

loss, accuracy = modelC.evaluate([X_test_ingredients, X_test_recipe_name, X_test_instructions], y_test_course)
print(f'Test Accuracy: {accuracy}')

Epoch 1/15
162/162 - 153s - loss: 0.1977 - accuracy: 0.2236 - val_loss: 0.1512 - val_accuracy: 0.2963 - 153s/epoch - 945ms/step
Epoch 2/15
162/162 - 121s - loss: 0.1337 - accuracy: 0.3971 - val_loss: 0.1201 - val_accuracy: 0.5134 - 121s/epoch - 749ms/step
Epoch 3/15
162/162 - 120s - loss: 0.1069 - accuracy: 0.5686 - val_loss: 0.1129 - val_accuracy: 0.5268 - 120s/epoch - 740ms/step
Epoch 4/15
162/162 - 120s - loss: 0.0938 - accuracy: 0.6408 - val_loss: 0.1127 - val_accuracy: 0.5279 - 120s/epoch - 741ms/step
Epoch 5/15
162/162 - 121s - loss: 0.0846 - accuracy: 0.6738 - val_loss: 0.1124 - val_accuracy: 0.5244 - 121s/epoch - 748ms/step
Epoch 6/15
162/162 - 115s - loss: 0.0781 - accuracy: 0.7039 - val_loss: 0.1130 - val_accuracy: 0.5285 - 115s/epoch - 711ms/step
Epoch 7/15
162/162 - 122s - loss: 0.0721 - accuracy: 0.7334 - val_loss: 0.1170 - val_accuracy: 0.5186 - 122s/epoch - 753ms/step
Epoch 8/15
162/162 - 116s - loss: 0.0661 - accuracy: 0.7541 - val_loss: 0.1199 - val_accuracy: 0.5297 - 

In [ ]:
with open('/content/drive/MyDrive/Model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)
with open('/content/drive/MyDrive/ModelC.pkl', 'wb') as model_file:
    pickle.dump(modelC, model_file)
with open('/content/drive/MyDrive/ModelCu.pkl', 'wb') as model_file:
    pickle.dump(modelCu, model_file)

In [ ]:
with open('/content/drive/MyDrive/mlb.pkl', 'wb') as f:
    pickle.dump(mlb, f)
with open('/content/drive/MyDrive/mlb_cuisine.pkl', 'wb') as f:
    pickle.dump(mlb_cuisine, f)
with open('/content/drive/MyDrive/mlb_course.pkl', 'wb') as f:
    pickle.dump(mlb_course, f)

# **Prediction**

In [4]:
with open('/content/drive/MyDrive/Model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)
with open('/content/drive/MyDrive/ModelC.pkl', 'rb') as model_file:
    modelC = pickle.load(model_file)
with open('/content/drive/MyDrive/ModelCu.pkl', 'rb') as model_file:
    modelCu = pickle.load(model_file)

with open('/content/drive/MyDrive/mlb.pkl', 'rb') as model_file:
    mlb = pickle.load(model_file)
with open('/content/drive/MyDrive/mlb_cuisine.pkl', 'rb') as model_file:
    mlb_cuisine = pickle.load(model_file)
with open('/content/drive/MyDrive/mlb_course.pkl', 'rb') as model_file:
    mlb_course = pickle.load(model_file)

In [5]:
df = pd.read_csv('/content/drive/MyDrive/IndianFoodDataset.csv')

In [7]:
max_words = 1500
tokenizer_ingredients = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer_ingredients.fit_on_texts(df['Ingredients'].astype(str))
tokenizer_recipe_name = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer_recipe_name.fit_on_texts(df['Name'].astype(str))
tokenizer_instructions = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer_instructions.fit_on_texts(df['Instructions'].astype(str))

In [8]:
new_input_ingredients = "2 Bay leaves (tej patta),1 Cardamom (Elaichi) Pods/Seeds,1 tablespoon Whole Black Peppercorns,5 Cloves (Laung),1 inch Cinnamon Stick (Dalchini),1 cup Red matta rice - washed thoroughly,Salt - to taste,1 inch Ginger,5 cloves Garlic,3 Green Chilli,300 grams Chicken - curry cut,2 Onions - sliced,3 Tomatoes - chopped,1 teaspoon Red Chilli powder,1/2 teaspoon Turmeric powder (Haldi),2 tablespoon Garam masala powder,1/4 cup Coriander (Dhania) Leaves - chopped,1/4 cup Mint Leaves (Pudina) - chopped,1 Lemon juice,1/2 cup Milk,5 Saffron strands,1 tablespoon Cashew nuts,1 tablespoon Ghee"
new_input_recipe_name = "Chicken Biryani"

new_sequences_ingredients = tokenizer_ingredients.texts_to_sequences([new_input_ingredients])
new_padded_ingredients = pad_sequences(new_sequences_ingredients, maxlen=150, padding='post', truncating='post')

new_sequences_recipe_name = tokenizer_recipe_name.texts_to_sequences([new_input_recipe_name])
new_padded_recipe_name = pad_sequences(new_sequences_recipe_name, maxlen=10, padding='post', truncating='post')

prediction = model.predict([new_padded_ingredients, new_padded_recipe_name])

threshold = 0.5
binary_prediction = (prediction > threshold).astype(int)
predicted_labels = mlb.inverse_transform(binary_prediction)

print("Predicted Labels:", predicted_labels)

1/1 [==============================] - 3s 3s/step
Predicted Labels: [('Non Vegeterian',)]


In [9]:
new_data_ingredients = "2 cups Basmati rice, 1 cup Chicken, 1 Onion, 2 Tomatoes, 1 tablespoon Ginger-Garlic paste, 1 teaspoon Garam masala, Salt to taste"
new_data_recipe_name = "Chicken Biryani"
new_data_instructions = "1. Wash the Basmati rice. 2. Marinate the chicken with ginger-garlic paste, garam masala, and salt. 3. Cook rice and chicken separately. 4. Mix them together."

sequences_new_ingredients = tokenizer_ingredients.texts_to_sequences([new_data_ingredients])
padded_sequences_new_ingredients = pad_sequences(sequences_new_ingredients, maxlen=150, padding='post', truncating='post')

sequences_new_recipe_name = tokenizer_recipe_name.texts_to_sequences([new_data_recipe_name])
padded_sequences_new_recipe_name = pad_sequences(sequences_new_recipe_name, maxlen=10, padding='post', truncating='post')

sequences_new_instructions = tokenizer_instructions.texts_to_sequences([new_data_instructions])
padded_sequences_new_instructions = pad_sequences(sequences_new_instructions, maxlen=200, padding='post', truncating='post')

predictions = modelC.predict([padded_sequences_new_ingredients, padded_sequences_new_recipe_name, padded_sequences_new_instructions])
binary_predictions = (predictions > 0.5).astype(int)
predicted_classes = mlb_course.inverse_transform(binary_predictions)

print("\nPredicted Classes:")
print(predicted_classes)

1/1 [==============================] - 4s 4s/step

Predicted Classes:
[()]


In [10]:
new_data_ingredients = "8 Hard taco shells,2 Potatoes (Aloo) - boiled and chopped,1/2 cup Onions - chopped,1/3 cup Tomatoes - chopped,2 tablespoons Sweet corn,1 tablespoon Green seedless Grapes - chopped,1 tablespoon Anardana Powder (Pomegranate Seed Powder),2 sprig Coriander (Dhania) Leaves - chopped,1 Green Chilli - chopped (adjustable),Chaat Masala Powder - to taste,Red Chilli powder - to taste,1/2 cup Sev,1/3 cup Boondi - spicy,1/2 cup Cheetos - crushed,1/2 cup Curd (Dahi / Yogurt) - mixed with 1 tablespoon sugar,1/3 cup Sweet Chutney (Date & Tamarind),1/3 cup Green Chutney (Coriander & Mint),3 tablespoons Lahsun Ki Chutney (Red Garlic Chutney)"
new_data_recipe_name = "taco chaat recipe"

sequences_new_ingredients = tokenizer_ingredients.texts_to_sequences([new_data_ingredients])
padded_sequences_new_ingredients = pad_sequences(sequences_new_ingredients, maxlen=150, padding='post', truncating='post')

sequences_new_recipe_name = tokenizer_recipe_name.texts_to_sequences([new_data_recipe_name])
padded_sequences_new_recipe_name = pad_sequences(sequences_new_recipe_name, maxlen=10, padding='post', truncating='post')

predictions = modelCu.predict([padded_sequences_new_ingredients, padded_sequences_new_recipe_name])

threshold = 0.5
binary_predictions = (predictions > threshold).astype(int)

mlb_cuisine_original = mlb_cuisine
mlb_cuisine_eval = MultiLabelBinarizer(classes=mlb_cuisine_original.classes_)
mlb_cuisine_eval.fit_transform([[]])
predicted_classes = mlb_cuisine_eval.inverse_transform(binary_predictions)

print("\nPredicted Classes:")
print(predicted_classes)

1/1 [==============================] - 3s 3s/step

Predicted Classes:
[('Fusion',)]


# **Result**

In [11]:
new_input_ingredients = "1 1/2 cups Whole Wheat Flour,3/4 cup Jaggery - powdered,2 1/2 cups Water,1/2 teaspoon Cardamom Powder (Elaichi),Ghee - for cooking the dosa', '1 कप सफेद उरद दाल\xa0(split),1 कप पिली मूंग दाल,1 कप अरहर दाल,2 कप चावल,1/4 कप मेथी के दाने,1 प्याज,3 हरी मिर्च,1 टहनी कढ़ी पत्ता,1/4 कप नारियल,2 छोटे चमच्च जीरा,नमक - स्वाद अनुसार,5 टहनी हरा धनिया"
new_input_recipe_name = "vellai / jaggerya dosa recipe"
new_data_instructions = "To begin making the Vellai Dosa, in a large bowl, dissolve the powdered jaggery along with 1/2 cup water and allow it to rest for 30 minutes. After 30 minutes, strain the jaggery water to remove impurities.Add the wheat flour to the strained jaggery mixture and stir continuously until there are no lumps. Add remaining 2 cups water and mix to make a smooth batter. Toss in the cardamom powder and mix well. The next step is to cook the Vella Dosa on a skillet. Heat a skillet on medium heat and pour a ladleful of batter on the skillet. Swirl the pan in a circular motion to spread the batter around to make a thin crepe like dosa.Drizzle 1/4 teaspoon of ghee around the vella dosa and cook on low heat until well done. Flip over to cook the other side for a few seconds and serve warm.Serve Vellai Dosa as a delicious dish during Navratri or as breakfast along with Peanut Chutney."

In [13]:
answer = []

new_sequences_ingredients = tokenizer_ingredients.texts_to_sequences([new_input_ingredients])
new_padded_ingredients = pad_sequences(new_sequences_ingredients, maxlen=150, padding='post', truncating='post')
new_sequences_recipe_name = tokenizer_recipe_name.texts_to_sequences([new_input_recipe_name])
new_padded_recipe_name = pad_sequences(new_sequences_recipe_name, maxlen=10, padding='post', truncating='post')
prediction = model.predict([new_padded_ingredients, new_padded_recipe_name])
threshold = 0.5
binary_prediction = (prediction > threshold).astype(int)
predicted_labels = mlb.inverse_transform(binary_prediction)
if(len(predicted_labels[0])):
  answer.append(predicted_labels[0][0])

sequences_new_ingredients = tokenizer_ingredients.texts_to_sequences([new_input_ingredients])
padded_sequences_new_ingredients = pad_sequences(sequences_new_ingredients, maxlen=150, padding='post', truncating='post')
sequences_new_recipe_name = tokenizer_recipe_name.texts_to_sequences([new_input_recipe_name])
padded_sequences_new_recipe_name = pad_sequences(sequences_new_recipe_name, maxlen=10, padding='post', truncating='post')
sequences_new_instructions = tokenizer_instructions.texts_to_sequences([new_data_instructions])
padded_sequences_new_instructions = pad_sequences(sequences_new_instructions, maxlen=200, padding='post', truncating='post')
predictions = modelC.predict([padded_sequences_new_ingredients, padded_sequences_new_recipe_name, padded_sequences_new_instructions])
binary_predictions = (predictions > 0.5).astype(int)
predicted_classes = mlb_course.inverse_transform(binary_predictions)
if(len(predicted_classes[0])):
  answer.append(predicted_classes[0][0])

sequences_new_ingredients = tokenizer_ingredients.texts_to_sequences([new_input_ingredients])
padded_sequences_new_ingredients = pad_sequences(sequences_new_ingredients, maxlen=150, padding='post', truncating='post')
sequences_new_recipe_name = tokenizer_recipe_name.texts_to_sequences([new_input_recipe_name])
padded_sequences_new_recipe_name = pad_sequences(sequences_new_recipe_name, maxlen=10, padding='post', truncating='post')
predictions = modelCu.predict([padded_sequences_new_ingredients, padded_sequences_new_recipe_name])
threshold = 0.5
binary_predictions = (predictions > threshold).astype(int)
mlb_cuisine_original = mlb_cuisine
mlb_cuisine_eval = MultiLabelBinarizer(classes=mlb_cuisine_original.classes_)
mlb_cuisine_eval.fit_transform([[]])
predicted_classes = mlb_cuisine_eval.inverse_transform(binary_predictions)
if(len(predicted_classes[0])):
  answer.append(predicted_classes[0][0])


print(answer)

1/1 [==============================] - 0s 54ms/step
['Vegetarian', 'South Indian Breakfast']


# **Searching for a Dish**

In [14]:
dish_names = df["Name"].str.lower().tolist()

new_dish_name = input("Enter the name of the dish you want to find a similar recipe for: ").lower()

vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(dish_names)
new_dish_vector = vectorizer.transform([new_dish_name])
similarities = cosine_similarity(new_dish_vector, tfidf_matrix)
top_n_indices = similarities.argsort()[0, -1:(-1 - 3):-1]

topDishes = []
most_similar_dishes = [dish_names[i] for i in top_n_indices]
print(f"The {3} most similar dishes in the dataset are:")
for dish in most_similar_dishes:
    topDishes.append(dish)
print(topDishes)

recipes = []
if(len(topDishes)):
  for DishDa in topDishes:
    recipe = df.loc[df["Name"].str.lower() == DishDa, "Ingredients"].values[0]
    recipes.append(recipe)
print(recipes)

Enter the name of the dish you want to find a similar recipe for: Dosa
The 3 most similar dishes in the dataset are:
['vellai / jaggerya dosa recipe', 'belle dosa recipe - lentil dosa', 'tomato dosa recipe']
['1 1/2 cups Whole Wheat Flour,3/4 cup Jaggery - powdered,2 1/2 cups Water,1/2 teaspoon Cardamom Powder (Elaichi),Ghee - for cooking the dosa', '1 कप सफेद उरद दाल\xa0(split),1 कप पिली मूंग दाल,1 कप अरहर दाल,2 कप चावल,1/4 कप मेथी के दाने,1 प्याज,3 हरी मिर्च,1 टहनी कढ़ी पत्ता,1/4 कप नारियल,2 छोटे चमच्च जीरा,नमक - स्वाद अनुसार,5 टहनी हरा धनिया', '1 cup rice, 1/4 cup white urad dal (split), 3 tomatoes, 1/4 teaspoon fenugreek seeds, salt - as per taste, 1/2 teaspoon cooking soda, water - as per use, oil - or ghee ( To make dosa)']


# **Final**

In [15]:
dish_names = df["Name"].str.lower().tolist()

new_dish_name = input("Enter the name of the dish you want to find a similar recipe for: ").lower()

vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(dish_names)
new_dish_vector = vectorizer.transform([new_dish_name])
similarities = cosine_similarity(new_dish_vector, tfidf_matrix)
top_n_indices = similarities.argsort()[0, -1:(-1 - 5):-1]

topDishes = []
most_similar_dishes = [dish_names[i] for i in top_n_indices]
print(f"The {5} most similar dishes in the dataset are:")
for dish in most_similar_dishes:
    topDishes.append(dish)
print("\n\nTop Dishes :\n")
print(topDishes)

recipes = []
inss = []
if(len(topDishes)):
  for DishDa in topDishes:
    recipe = df.loc[df["Name"].str.lower() == DishDa, "Ingredients"].values[0]
    ins = df.loc[df["Name"].str.lower() == DishDa, "Instructions"].values[0]
    recipes.append(recipe)
    inss.append(ins)
print("\n\nRecipes :\n")
print(recipes)
print(inss)

Enter the name of the dish you want to find a similar recipe for: Dosa
The 5 most similar dishes in the dataset are:


Top Dishes :

['vellai / jaggerya dosa recipe', 'belle dosa recipe - lentil dosa', 'tomato dosa recipe', 'tomato dosa recipe', 'egg dosa recipe']


Recipes :

['1 1/2 cups Whole Wheat Flour,3/4 cup Jaggery - powdered,2 1/2 cups Water,1/2 teaspoon Cardamom Powder (Elaichi),Ghee - for cooking the dosa', '1 कप सफेद उरद दाल\xa0(split),1 कप पिली मूंग दाल,1 कप अरहर दाल,2 कप चावल,1/4 कप मेथी के दाने,1 प्याज,3 हरी मिर्च,1 टहनी कढ़ी पत्ता,1/4 कप नारियल,2 छोटे चमच्च जीरा,नमक - स्वाद अनुसार,5 टहनी हरा धनिया', '1 cup rice, 1/4 cup white urad dal (split), 3 tomatoes, 1/4 teaspoon fenugreek seeds, salt - as per taste, 1/2 teaspoon cooking soda, water - as per use, oil - or ghee ( To make dosa)', '1 cup rice, 1/4 cup white urad dal (split), 3 tomatoes, 1/4 teaspoon fenugreek seeds, salt - as per taste, 1/2 teaspoon cooking soda, water - as per use, oil - or ghee ( To make dosa)', '2 c

In [16]:
answer = []
if(len(topDishes) >= 5 and len(recipes) >= 5):

  for i in range(0, 5):

    new_input_ingredients = topDishes[i]
    new_input_recipe_name = new_dish_name
    new_data_instructions = inss[i]

    new_sequences_ingredients = tokenizer_ingredients.texts_to_sequences([new_input_ingredients])
    new_padded_ingredients = pad_sequences(new_sequences_ingredients, maxlen=150, padding='post', truncating='post')
    new_sequences_recipe_name = tokenizer_recipe_name.texts_to_sequences([new_input_recipe_name])
    new_padded_recipe_name = pad_sequences(new_sequences_recipe_name, maxlen=10, padding='post', truncating='post')
    prediction = model.predict([new_padded_ingredients, new_padded_recipe_name])
    threshold = 0.5
    binary_prediction = (prediction > threshold).astype(int)
    predicted_labels = mlb.inverse_transform(binary_prediction)
    if(len(predicted_labels[0])):
      answer.append(predicted_labels[0][0])

    sequences_new_ingredients = tokenizer_ingredients.texts_to_sequences([new_input_ingredients])
    padded_sequences_new_ingredients = pad_sequences(sequences_new_ingredients, maxlen=150, padding='post', truncating='post')
    sequences_new_recipe_name = tokenizer_recipe_name.texts_to_sequences([new_input_recipe_name])
    padded_sequences_new_recipe_name = pad_sequences(sequences_new_recipe_name, maxlen=10, padding='post', truncating='post')
    sequences_new_instructions = tokenizer_instructions.texts_to_sequences([new_data_instructions])
    padded_sequences_new_instructions = pad_sequences(sequences_new_instructions, maxlen=200, padding='post', truncating='post')
    predictions = modelC.predict([padded_sequences_new_ingredients, padded_sequences_new_recipe_name, padded_sequences_new_instructions])
    binary_predictions = (predictions > 0.5).astype(int)
    predicted_classes = mlb_course.inverse_transform(binary_predictions)
    if(len(predicted_classes[0])):
      answer.append(predicted_classes[0][0])

    sequences_new_ingredients = tokenizer_ingredients.texts_to_sequences([new_input_ingredients])
    padded_sequences_new_ingredients = pad_sequences(sequences_new_ingredients, maxlen=150, padding='post', truncating='post')
    sequences_new_recipe_name = tokenizer_recipe_name.texts_to_sequences([new_input_recipe_name])
    padded_sequences_new_recipe_name = pad_sequences(sequences_new_recipe_name, maxlen=10, padding='post', truncating='post')
    predictions = modelCu.predict([padded_sequences_new_ingredients, padded_sequences_new_recipe_name])
    threshold = 0.5
    binary_predictions = (predictions > threshold).astype(int)
    mlb_cuisine_original = mlb_cuisine
    mlb_cuisine_eval = MultiLabelBinarizer(classes=mlb_cuisine_original.classes_)
    mlb_cuisine_eval.fit_transform([[]])
    predicted_classes = mlb_cuisine_eval.inverse_transform(binary_predictions)
    if(len(predicted_classes[0])):
      answer.append(predicted_classes[0][0])

print(answer)
answer = list(set(answer))
print(answer)

1/1 [==============================] - 0s 52ms/step
['Vegetarian', 'South Indian Breakfast', 'South Indian Recipes', 'Vegetarian', 'South Indian Breakfast', 'South Indian Recipes', 'Vegetarian', 'South Indian Breakfast', 'South Indian Recipes', 'Vegetarian', 'South Indian Breakfast', 'South Indian Recipes', 'Vegetarian', 'South Indian Breakfast', 'South Indian Recipes']
['South Indian Recipes', 'South Indian Breakfast', 'Vegetarian']
